# Simulate synapses onto aCC

In [ ]:
%matplotlib notebook

import csv
from neuron import h, gui
from neuron.units import ms, mV
from matplotlib import pyplot

## Load morphology and other components

In [ ]:
# load channels
h('''{load_file("../chan-DmNaT-ODowd.hoc")
      load_file("../chan-DmNaP-DmNav10.hoc")
      load_file("../chan-DmKdr-Marley.hoc")
      load_file("../chan-DmKA-Marley.hoc")
     }''')

# load morphology
h('{load_file("../neuron-morph-ext-axon-2pieces-chans-ext-axon-70um.hoc")}')

# enable or disable voltage clamp
h('load_file("../vclamp_soma_-60mV.ses")')

## Initialization

In [10]:
# load common funcs
h('load_file("../fitfuncs.hoc")')

#h('print_elec_cell()') #fix this 

# calc morph stats
h('load_file("../stats.hoc")')

file_name = "ext-axon-70um-sim-synapses"

# save state in this file
h.state_file_name = "state-neuron-act+elec+ext-axon-2piece-chans-" + file_name + ".bin"

# load state first because it overrides parameters
state_w_syn = "state-neuron-act+elec+ext-axon-2piece-chans-" + file_name + "-synapse-saved.bin"
h.restoreStateFromFile(state_w_syn)

# small adjustments

# increase VC duration
h.VClamp[0].dur[0] = 1000

	1 
	1 


In [ ]:
# Dictionary for netcon objects indexed by section number that hold the synapse and netstim objects
syns = {}

class Synapse():
    
    def __init__(self, name, sec, syn, ns, nc):
        self.name = name
        self.sec = sec
        self.syn = syn
        self.ns = ns
        self.nc = nc
        
    def set_stim(self, ps, weight = .00018, tau = 5, interval = 10, magepsc = 20, syne = 0, number = 1, start = 10):
        """Set parameters of specified synapse"""
        nc = self.nc
        syn = self.syn
        ns = self.ns
        print(f"Setting: weight={weight}, tau={tau}, interval={interval}, magepsc={magepsc}, syne={syne}, number={number}, start={start}")
        syn.tau = tau
        syn.e = syne
        ns.interval = interval
        ns.number = number
        ns.start = start
        ns.noise = 0
        nc.weight[0] = weight * magepsc
        ps.color(2, self.sec(0.5))

def create_one_syn(sec):
    """Factory method to create synapse, NetStim, and NetCon objects"""
    print(f"Creating synapse on {sec}\n")
    syn = h.ExpSyn(sec(0.5))
    ns = h.NetStim(sec(0.5))
    nc = h.NetCon(ns, syn)
    syn_obj = Synapse(str(sec), sec, syn, ns, nc)
    syns[str(sec)] = syn_obj
    return syn_obj

'''
 * synapses:
 * top dend: 685, 524, 520, 626
 * bot dend: 205, 357, 464, 588, 513, 48
'''

#create_syn_dendrite_section(685, 524, 520, 626, 205, 357, 464, 588, 513, 48)
#set_syn_pars()                  # use defaults

# Experiments

## create shape plot

In [ ]:
ps = h.PlotShape(0) # don't show a default view
ps.show(0)         # show diameters
ps.view(30,10,55,87, 300, 100, 500, 700)
h.load_file('stdrun.hoc')

## stimulate one synapse with varying magnitudes

In [ ]:
syn = create_one_syn(h.dendrite[685])

In [ ]:
syn.set_stim(ps = ps, weight = .0018, tau=5, interval = 0, number = 1)
h.finitialize(-65 * mV)
#h.continuerun(60 * ms)
h.tstop = 60 * ms
h.run()

In [ ]:
syn2 = create_one_syn(h.dendrite[48])

In [ ]:
#syn.set_stim(ps = ps, weight = 0, tau=5, interval = 0, number = 1)
syn2.set_stim(ps = ps, weight = .0018, tau=5, interval = 0, number = 1)
h.finitialize(-65 * mV)
h.continuerun(60 * ms)
#h.tstop = 60 * ms
#h.run()

## Save state

In [11]:
print(state_file_name)
h.finitialize()
h.saveState()

state-neuron-act+elec+ext-axon-2piece-chans-ext-axon-70um-sim-synapses.bin


0.0

In [8]:
h.steady_state.save()
h('state_fobj = new File("state-neuron-act+elec+ext-axon-2piece-chans-ext-axon-70um-sim-synapses.bin")')
h.steady_state.fwrite(h.state_fobj)

1.0